# Enunciado


Este trabalho usa SageMath nas suas implementações


1. Pretende-se construir em torno de uma cifra assimétrica um conjunto de técnicas criptográficas destinadas a fins distintos. Apesar de todas as alíneas do problema poderem ser  respondidas com a maioria das cifras assimétricas clássicas ou pós-quânticas, neste problema vamos exemplificar o processo com uma técnica simples da família Diffie-Hellman nomeadamente a cifra assimétrica ElGamal com parâmetros de segurança $\,\lambda\,$.
    1. Implemente um esquema  PKE $\,\mathsf{ElGamal}(\lambda)\,$ (ver Capítulo 4) num subgrupo de ordem prima $\,q\,$,  com $\,|q|\geq \lambda\,$, do grupo multiplicativo $\,\mathbb{F}^\ast_p\,$ com $\,p\,$ um primo que verifica $\,|p| \geq \lambda\times|\lambda|$ . Identifique o gerador de chaves e os algoritmos de cifra de decifra neste esquema. Identifique o núcleo deterministico do algoritmo de cifra.
    2. Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.
    3. A partir de (b) construa um esquema de KEM que seja IND-CCA seguro.
    4. A partir de (b) construa uma implementação de um protocolo autenticado de "Oblivious Transfer" $\,\kappa$-out-of-$n\,$.

# **Exercício 1.a.**

EL GAMAL
Qualquer PKE é determinado por três algoritmos: geração de chaves, cifra e decifra:

 ----------<br><br>$\text{GenKeys}(\lambda)$                                 …       $\lambda\;$ é o parâmetro de segurança<br><br><br>    - gerar aleatoriamente um primo $\,q \approx 2^\lambda$                                      <br>    - gerar um primo $p$  tal que  $\,\mathbb{F}_p^\ast\,$ tem um sub-grupo de ordem $\,q\,$ ; calcular um gerador $g$ desse sub-grupo<br>    - gerar aleatoriamente  $\,0 <s < q\,$ ,  a chave privada<br>    - calcular e  revelar  a chave pública   $\,\mathsf{pk} \equiv \langle p,q, g,g^s\rangle$<br>----------<br><br>$\text{Enc}(\mathsf{pk},m)$                                   …   a mensagem $m$ é um elemento de $\mathbb{F}_p^\ast$ <br><br><br>    - obter elementos públicos  $\,p,q,g,g^s \,\gets\,\mathsf{pk}$<br>    - gerar aleatoriamente  $\,0 <\omega < q$ <br>    - calcular  $\,\gamma \gets g^\omega\;$ e $\,\kappa \gets (g^s)^\omega\,$.<br>    - construir  o criptograma $\,\mathbf{c}\gets \langle\,\gamma\,,\, m\times\kappa\,\rangle\,$<br>----------<br><br>Note-se que se verifica $\,\kappa = \gamma^s\,$.<br> 
----------<br><br>$\text{Dec}(\mathsf{sk},\mathbf{c})$  …  $\mathsf{sk} = s$ é a chave privada<br><br>
- obter a chave privada $s$<br>    
- obter o criptograma $\mathbf{c} = \langle \gamma, \delta \rangle$<br>    
- calcular $\kappa \gets \gamma^s \mod p$<br>    
- calcular $\kappa^{-1} \mod p$<br>    
- recuperar a mensagem original: $m \gets \delta \times \kappa^{-1} \mod p$<br>    

In [1]:
print_tentativas_genKeys = False

In [2]:
from sage.all import *
lambda_security = 128  # Define um tamanho de bits para q

Para encontrar um gerador do grupo multiplicativo temos que encontrar um número cuja ordem seja igual à do grupo, i.e.:
- Dada a ordem do grupo multiplicativo $F_p^* = \phi(p)$ o gerador g tem de ter ordem $\phi(p)$, ou seja, $g^{\phi(n)}=1 \ mod \ p$

In [3]:
def find_generator(p):
    """Encontra um gerador do grupo multiplicativo F_p^*."""
    if not is_prime(p):
        raise Exception("O p de input não é primo")

    phi_p = p - 1  # Para p primo, phi(p) = p - 1

    fatoracao = factor(phi_p)
    fatores_primos = list(set([q for q, e in fatoracao]))
    
    print(f"Fatoração de phi(p): {fatoracao}")
    print(f"Fatores primos de phi(p): {fatores_primos}")

    # Itera sobre possíveis geradores
    for g in range(2, p):
        if gcd(g, p) != 1:
            continue  # Ignora números não coprimos com p

        is_gerador = True
        for q in fatores_primos:
            if pow(g, phi_p // q, p) == 1:
                is_gerador = False
                break

        if is_gerador:
            return g 

    return None  

In [4]:
def gen_keys(lambda_security):
    print("lambda: ",lambda_security)
    # Gerar aleatoriamente q com bit_length maior que lambda
    q = random_prime(2^129 - 1, False, 2^128)
    print("Parâmetro q gerado:",q)
    print("Tamanho de q:",q.nbits())
    
    #Gera-se sucessivamente inteiros pi = q*2^i+1 até que pi seja um primo suficientemente grande, ou seja |p| > 1024
    i = 1
    p_i = q * (2^i) + 1 
    tamanho_desejado = lambda_security * lambda_security.nbits()
    
    while True:
        # Caso o p_i não convergir tenta um novo q
        if p_i.nbits() > 2500:
            q = random_prime(2^129 - 1, False, 2^128)
            i = 1
            p_i = q * (2^i) + 1
            print("p não convergiu, novo valor de q: ",q)
            
        if is_prime(p_i) and p_i.nbits() >= tamanho_desejado:
            break  
    
        i += 1
        p_i = q * (2^i) + 1
        if print_tentativas_genKeys:
            print(f"Tentativa {i}: p_i = {p_i} (Tamanho: {p_i.nbits()} bits)")

    p = p_i
    print("Parâmetro p gerado:",p)
    print(f"Tamanho de bits de p (lambda x |lambda| = {tamanho_desejado}): {p.nbits()}")
    
    # Fp é um grupo multiplicativo se para todo o x pertencente a Fp gcd(x,p) = 1 (trivial já que p é primo)
    # Criar o corpo finito F_p
    F_p = GF(p)
    # O grupo multiplicativo F_p^* é o conjunto de elementos não nulos de F_p
    F_p_star = F_p.unit_group()
        
    # Agora obtemos um gerador do subgrupo de ordem q
    g = find_generator(p)
    print("g:",g)
    
    # Gerar aleatoriamente a chave privada 0<s<q
    s = randint(1, q-1)
    
    return (p, q, g, pow(g, s, p)), s

## Escolha de p:
A ordem desse grupo é $\,n = p - 1\,$ e para que o DLP seja  complexo não basta apenas que $\,p\,$ seja grande: é também necessário que o maior factor primo de $\,(p-1)\,$ seja também grande. 
Para garantir estas condições o primo $\,p\,$  é gerado de uma determinada forma:

1. Gera-se um primo $\,q\,$ grande: com mais de $\lambda$ bits de tamanho; este vai ser o maior factor de $\,(p-1)\,$
2. Gera-se sucessivamente inteiros  $\,p_i\;=\;q\,2^i + 1\,$ até que $\,p_i\,$ seja  um primo suficientemente grande .

## Como é que $F_p^*$ tem um subgrupo de ordem q? (Teorema de Lagrange)
O Teorema de Lagrange diz que, se um grupo G tem ordem finita e H é um subgrupo de G, então |H| é um divisor de |G|.

Visto que $F_p^*$ tem ordem $p-1$ e $p-1$ divide $2q$ (e consequentemente $q$) então podemos afirmar que $F_p^*$ tem um subgrupo de ordem q. 

------------------

In [5]:
def enc(pk, m):
    p, q, g, g_s = pk
    omega = randint(1, q-1)  # Escolhe um valor aleatório ω entre 0 e q
    gamma = pow(g, omega, p)  # γ = g^ω mod p
    kappa = pow(g_s, omega, p)  # κ = (g^s)^ω mod p
    c = (gamma, (m * kappa) % p)  # Criptograma
    return c

------------------

In [6]:
def dec(sk, pk, c):
    p, _, _, _ = pk
    gamma, delta = c
    kappa = pow(gamma, sk, p)  # κ = γ^s mod p
    kappa_inv = inverse_mod(Integer(kappa), Integer(p))  # Calcula o inverso de κ mod p
    m = (delta * kappa_inv) % p  # Recupera a mensagem original
    return m

------------------

In [7]:
# Exemplo
pk, sk = gen_keys(lambda_security)  # Geração de chaves
m = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
c = enc(pk, m)  # Cifra a mensagem
m_dec = dec(sk, pk, c)  # Decifra a mensagem

# Exibir os resultados
print(f"Mensagem original: {m}")
print(f"Criptograma: {c}")
print(f"Mensagem decifrada: {m_dec}")
print(f"Decifração bem-sucedida? {m == m_dec}")

lambda:  128
Parâmetro q gerado: 403375798132636591563995377708191291601
Tamanho de q: 129
Parâmetro p gerado: 2478243596982939790757749352506836421525691407195082399551414017424166442915789393326074074263611252513777814596783983686248175279086242963896781948441310817474452207868506082330789662051309677707060473962158669769139919542938121260615916489710146003308346215418994757803064628039448590942490429656892966615315512600566349591080249640786443960090031140284414797001717921801978083856901681865059517905774837818663995780227181048162669604955488257
Tamanho de bits de p (lambda x |lambda| = 1024): 1477
Fatoração de phi(p): 2^1348 * 403375798132636591563995377708191291601
Fatores primos de phi(p): [2, 403375798132636591563995377708191291601]
g: 3
Mensagem original: 15467236432901512961553628179058685185458371223578127068087288594185508508009342661818254225123666015799069919372043278204685896823221108110889402780537080755590635793764313448048317479944724088271046601572489976261413678419

--------

# **Exercício 1.b.**

Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$

### Definições de variáveis

In [8]:
print("----------------------------------Definição de veriáveis----------------------------------")
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')
print("length x:",len(x))
print("Mensagem de input:",x)
print("-------------------------------------------------------------------------------------------")

----------------------------------Definição de veriáveis----------------------------------
lambda:  128
Parâmetro q gerado: 658718539952579972405075393153779598417
Tamanho de q: 129
p não convergiu, novo valor de q:  363304254277967637575396774975950025911
p não convergiu, novo valor de q:  588092000229710790720553713739672430861
p não convergiu, novo valor de q:  669109259562764449524966804068993842343
Parâmetro p gerado: 71003915862298958125476879588726389593795749008826188513955551097652390101047068759254794513381905521414619911540090237611917652604537805889497255887390898744805448642896614223934539633177359497300377432268404575030514691206619163650265926858862431719699960209577451332714040079446487687716480119090444051812303739013848234803297849199709636019028270802633405975494657
Tamanho de bits de p (lambda x |lambda| = 1024): 1222
Fatoração de phi(p): 2^1093 * 669109259562764449524966804068993842343
Fatores primos de phi(p): [669109259562764449524966804068993842343, 2]
g: 3
leng

-------

### Funções auxiliares:
- $g$, um "hash" pseudo-aleatório de tamanho $|x|$
- $h$, um "hash" pseudo-aleatório de tamanho $lambda$
- $f_p$ o núcleo determinístico da cifra ElGamal

In [9]:
import hashlib

def g(r):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    g.update(r_bytes)
    final_hash = g.digest()  # Truncar para o tamanho de x
    while len(final_hash) < len(x):
        g = hashlib.sha512()
        g.update(r_bytes)
        final_hash += g.digest()
    print("g(r) hash:", final_hash[:len(x)])
    return final_hash[:len(x)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    print("h(r, y) hash:", full_hash)
    return full_hash


#Núcleo determinístico da função enc anterior
def f_p(pk, r, rlinha):
    p, q, g, g_s = pk
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p
    return (gamma, (r * kappa) % p)

### Cifra IND-CPA segura 

In [10]:
print("----------------------------------CIFRA----------------------------------")
def enc_fujisaki(pk,x):
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)
    a = g(r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))
    rlinha = h(r,y)
    c = f_p(pk,r,int.from_bytes(rlinha,"big"))
    return (y,c)

(y,c) = enc_fujisaki(pk,x)
print("(y,c) = ",(y,c))
print("-------------------------------------------------------------------------------------------")

----------------------------------CIFRA----------------------------------
g(r) hash: b'\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|'
g(r) hash: b'\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0

### Decifra

In [11]:
print("----------------------------------DECIFRA----------------------------------")
def dec_fujisaki(sk,pk,y,c):
    r = dec(sk,pk,c)
    rlinha = h(r,y)
    if c != f_p(pk,r,int.from_bytes(rlinha,"big")):
        raise Exception("O criptograma não corresponde a fp(r,r'), absurdo")
    else:
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

print("Mensagem decifrada: ",dec_fujisaki(sk,pk,y,c))
print("x == dec_fujisaki(sk,pk,y,c)?: ",x == dec_fujisaki(sk,pk,y,c)) 
print("-------------------------------------------------------------------------------------------")

----------------------------------DECIFRA----------------------------------
h(r, y) hash: b'7\xba3\x8f\xcaq\xcd%\xac y<\xd0\xe4\xd6\xe5'
g(r) hash: b'\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|\x02\x05s^\xa49\x8fQ\xaf\xbb\x94\x86a.\x83\xae\'\xccf\xb9\xf7\xf9\xa3\xcb-\x9a\xaf\xa4(\x97u"#\xdf\xfd\xf3\xe7\x8a\x9a\xedq\x9d\xa9\x1e\xf7\xe0T\x8e\xc4\xd4e\xb8\x0e(\x81\xb2\x7f\x18\xbd_S\xdet|'
Mensagem decifrada:  b'=\xf6\xfd\x19\x86\x83\x17\x9c?\x08\xc6N\xf9\x1f\x900J\xa32wH\x12\xc6+\xfdP\x9e!\xc1\xb0a\xd3\xac*\xf6\x18+\xc2\x8b\xc1_\x12\xd9+\xa9;Z\n\xfb.\x89eJ\xbc\x82\xdd=\xe7\xa9\xd8\xe1\x17\xfbM\x11\xcf\xc0\xf5\xf1\xf1px\x8d\xd4\xb5a\x0e\xdb\x87\xed\x04\xd4\xd7o4\xcc\x91V6\xdc}P\x1bFUg\x11\xe1\x0e\xa1\xefL\x93\x10\xcao:r\xf4\x0f\xa0\xb0u\x8e\x88 \x1f\xba\x0b\xe9\xa4\x